# Distance
Define distance (RNA1, RNA2, K) as:   
Euclidean distance between K-mer count vectors of RNA1 and RNA2.

In MLP 15, we examined the 2 mRNA sequences with minimal distance.
These turned out to be from the same gene: 
a normal gene called CENPS, 
plus its overlapping "readthrough" called CENPS-CORT.
(Why does Ensembl consider these different genes???)

We may need to compute all-vs-all pairwise distances.
That will require HPC.

Much less expensive would be all-vs-mean distances.
In this notebook, we try to find the mean K-mer vectors
for nuclear vs cytoplasmic, 
within the training set mRNA, at K=4.

In [1]:
from datetime import datetime
print(datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

2022-10-23 13:48:31.286768
Python 3.10.0
sklearn 1.1.2


In [2]:
from KmerCounter import KmerCounter
K=4
counter=KmerCounter(K)
VOCABULARY_SIZE = counter.get_vocabulary_size() 
from cell_lines import Cell_Lines
CELL_LINE_NUMBER = 0
all_cell_lines = Cell_Lines.get_ordered_list()
cell_line_name = all_cell_lines[CELL_LINE_NUMBER]
print('Cell line for today:',CELL_LINE_NUMBER,'=',cell_line_name)

Cell line for today: 0 = A549


In [3]:
ATLAS_DIR = '/Users/jasonmiller/WVU/Localization/LncAtlas/'
RCI_FILE = 'CNRCI_coding_train_genes.csv'
GENCODE_DIR = '/Users/jasonmiller/WVU/Localization/GenCode/'
COUNTS_FILE='CNRCI_coding_train_counts.K4.csv'

In [4]:
from TrainValidSplit2 import Splitter2
splitter = Splitter2()
ATLAS_PATH = ATLAS_DIR+RCI_FILE
gene_rci = splitter.get_gene_universe(ATLAS_PATH,CELL_LINE_NUMBER)
COUNTS_PATH = GENCODE_DIR+COUNTS_FILE
gid_tid,ordered_counts = splitter.load_counts_universe(COUNTS_PATH)
print
splitter = None

Loaded RCI values for cell line 0
Selected 10354 values out of 13930 genes.
Loaded 67964 gid+tid combinations.
Loaded 67964 rows of K-mer counts.


Ideas:

Narrow the RNA set to just extremes, Gudenas-style.

The following vectors have 256 counts.

Compute the mean vector for high and for low RCI.

Compute vector of presence/absence i.e. sequences with this K-mer.

Compute mean & stdev distance to the mean vector.

Are nuclear RNA more like the nuclear mean vector?
